# Example: Elliptical Orbit

## Problem Statement

A geocentric elliptical orbit has a perigee radius of 9600 km and an apogee radius of 21,000 km. Calculate the time to fly from perigee to a true anomaly of $\theta =$ 120°. Then, calculate the true anomaly 3 hr after perigee.

## Solution

The first step in these problems is always to find the primary orbital parameters, $h$ and $e$. To obtain $e$, we can use the equation in terms of the distances to perigee and apogee, Eq. {eq}`ellipse-eccentricity-periapsis-apoapsis`:

$$e = \frac{r_a - r_p}{r_a + r_p}$$

In [1]:
import numpy as np
from scipy.optimize import newton

mu = 3.986004418E5 # km**3/s**2
theta = np.radians(120)

In [2]:
r_p = 9600  # km
r_a = 21_000 # km
e = (r_a - r_p)/(r_a + r_p)
print(round(e, 4))

0.3725


Then, the specific angular momentum is obtained from the orbit equation, solved for either the apogee or perigee radius:

$$h = \sqrt{r_p \mu \left(1 + e\right)}$$

In [3]:
h = np.sqrt(r_p * mu * (1 + e))
print(round(h, 2))

72471.7


Now, to find the time to fly to the true anomaly of 120°, we need to find $M_e$. $M_e$ is found from the eccentric anomaly, $E$, which is found from $\theta$:

$$E = 2 \tan^{-1}\left(\sqrt{\frac{1 - e}{1 + e}}\tan\frac{\theta}{2}\right)$$

In [4]:
E = 2 * np.arctan(np.sqrt((1 - e)/(1 + e)) * np.tan(theta / 2))
print(round(E, 3), f"{np.degrees(E):.3F}°")

1.728 99.011°


Then, the mean anomaly is found from Kepler's equation:

$$M_e = E - e\sin E$$

In [5]:
M_e = E - e * np.sin(E)
print(round(M_e, 3))

1.36


Finally, calculating the time from the mean anomaly requires the period. The period requires the semimajor axis, $a$:

$$a = \frac{r_p}{1 - e}$$

and

$$T = \frac{2\pi}{\sqrt{\mu}}a^{3/2}$$

or Kepler's third law. Then, the time to the true anomaly of 120° is:

$$t = \frac{M_e T}{2 \pi}$$

In [6]:
a = r_p / (1 - e)
T = 2 * np.pi / np.sqrt(mu) * a**(3 / 2)
t = M_e * T / (2 * np.pi)
print(a, T, t, t / 3600)

15300.0 18834.24114907306 4077.043054361004 1.1325119595447233


The total time is 4,077 seconds or just over 1 hour.

Now, let's calculate the true anomaly about 2 hours later, after 3 total hours since perigee have elapsed. Since we already have the orbital eccentricity and specific angular momentum, we can start by finding the mean anomaly after 3 hours.

In [7]:
t = 3  # hr
M_e = 2 * np.pi * t * 3600 / T
print(round(M_e, 3))

3.603


Now, we need to solve Kepler's equation to find the eccentric anomaly, $E$. Since the equation is transcendental in $E$, we need to use the Newton solver in SciPy. Since we know the derivative, we will define two Python functions:

1. Kepler's equation, $f(E) = 0$
2. The derivative of Kepler's equation with respect to $E$, $f'(E)$

In [8]:
def kepler(E, M_e, e):
    """Kepler's equation, to be used in a Newton solver."""
    return E - e * np.sin(E) - M_e

def d_kepler_d_E(E, M_e, e):
    """The derivative of Kepler's equation, to be used in a Newton solver.
    
    Note that the argument M_e is unused, but must be present so the function
    arguments are consistent with the kepler function.
    """
    return 1 - e * np.cos(E)

E = newton(func=kepler, fprime=d_kepler_d_E, x0=np.pi, args=(M_e, e))
print(round(E, 3))

3.479


With this value for $E$, we can calculate the value for $\theta$. To avoid sign problems, we will use the tangent formula:

$$\tan\frac{\theta}{2} = \sqrt{\frac{1 + e}{1 - e}} \tan\frac{E}{2}$$

or

$$\theta = 2\tan^{-1}\sqrt{\frac{1 + e}{1 - e}} \tan\frac{E}{2}$$

In most programming languages, Python and MATLAB included, the `arctan` function returns a value between $-\pi/2$ and $\pi/2$. When the result is multiplied by 2, it gives the range from $-\pi$ to $\pi$. We need to transform this angle to be in the range of $0$ to $2\pi$. To do so, we can take the **modulus** of the angle with $2\pi$.

The modulus is the remainder after division. In Python, the modulus operator is `%`, while in MATLAB, we have to use the function `mod(numerator, denominator)`. This works for both positive and negative numbers, and ensures that we get the correct angle for the appropriate quadrant.

Note that this means we do not need to use the `arctan2` function, which we can't really use anyways since we have an angle instead of two points in the coordinate system.

In [9]:
theta = (2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))) % (2 * np.pi)
print(round(theta, 3), f"{np.degrees(theta):.3F}°")

3.371 193.156°
